# NetworkX_Tutorial

How to use networkX + loads of learning about network analysis.

### Import

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import pandas as pd
import numpy as np
import apiIntegrations.ga
import topicmodelling.utilities.clean
import topicmodelling.classes

### Import Data

In [ ]:
edges = pd.read_csv(r"C:\Users\Tobias Fechner\Documents\1_Uni\fyp\git_repo_fyp\data\networktheory\sample_data\quakers_edgelist.csv")
nodes = pd.read_csv(r"C:\Users\Tobias Fechner\Documents\1_Uni\fyp\git_repo_fyp\data\networktheory\sample_data\quakers_nodelist.csv")

In [ ]:
nodes.head()

In [ ]:
edges.head()

### Import and start using NetworkX

In [ ]:
import networkx as nx

Create a graph object

In [ ]:
g = nx.Graph()

Add the nodes and edges from lists

In [ ]:
g.add_nodes_from(nodes.loc[:, 'Name'].values)

In [ ]:
g.add_edges_from(list(zip(edges.Source, edges.Target)))

Print some info

In [ ]:
print(nx.info(g))

Now we add some attributes. These need to be a dictionary for each attribute.

In [ ]:
nodes.head(1)

Write a function to create attributes dictionaries

In [ ]:
def createRowAttributes(row):
    name = row.iloc[0]
    
    hist_sig = ('hist_sig', {name: row.iloc[1]})
    gender = ('gender', {name: row.iloc[2]})
    birth = ('birth', {name: row.iloc[3]})
    death = ('death', {name: row.iloc[4]})
    row_id = ('id', {name: row.iloc[5]})
    
    return hist_sig, gender, birth, death, row_id

Apply to the dataframe

In [ ]:
nodes['attributes'] = nodes.apply(lambda x: createRowAttributes(x), axis=1)
nodes.head()

Apply function to add each attribute to the graph for each row in the dataframe. First create a function to add the attributes.

In [ ]:
def addAttributesToGraph(graph, attributes):
    for attribute in attributes:
        nx.set_node_attributes(graph, values=attribute[1], name=attribute[0])

Then apply this function down the column to add each row's attributes to the graph.

In [ ]:
nodes['attributes'].apply(lambda x: addAttributesToGraph(g, x))

In [ ]:
for n in list(g.nodes())[:10]: # Loop through first 10 nodes, in our data "n" will be the name of the person
    print(n, g.nodes[n]['birth']) # Access every node by its name, and then by the attribute "birth" - birth year

# Inspect the Network: Generate some Metrics

### Structural Calculations

These metrics look at the entire network as a whole.

##### Network density

A good metric to begin with is network density. This is simply the ratio of actual edges in the network to all possible edges in the network. In an undirected network like this one, there could be a single edge between any two nodes, but as you saw in the visualization, only a few of those possible edges are actually present. Network density gives you a quick sense of how closely knit your network is.

In [ ]:
density = nx.density(g)
print(f"Network density: {round(density, 6)}")

##### Shortest path

To calculate a shortest path, you’ll need to pass several input variables (information you give to a Python function): the whole graph, your source node, and your target node.

Depending on the size of your network, this could take a little while to calculate, since Python first finds all possible paths and then picks the shortest one. 

In [ ]:
fell_whitehead_path = nx.shortest_path(g, source="Margaret Fell", target="George Whitehead")

print("Shortest path between Fell and Whitehead:", fell_whitehead_path)

##### Diameter

There are many network metrics derived from shortest path lengths. One such measure is diameter, which is the longest of all shortest paths. 

Since there is no shortest path between nodes of one component and nodes of another, nx.diameter() returns the “not connected” error. You can remedy this by first finding out if your Graph “is connected” (i.e. all one component) and, if not connected, finding the largest component and calculating diameter on that component alone.

In [ ]:
# If your Graph has more than one component, this will return False:
print(nx.is_connected(g))

# Next, use nx.connected_components to get the list of components,
# then use the max() command to find the largest one:
components = nx.connected_components(g)
largest_component = max(components, key=len)

# Create a "subgraph" of just the largest component
# Then calculate the diameter of the subgraph, just like you did with density.
#

subgraph = g.subgraph(largest_component)
diameter = nx.diameter(subgraph)
print("Network diameter of largest component:", diameter)

##### Triadic Closure

Triadic closure supposes that if two people know the same person, they are likely to know each other. If Fox knows both Fell and Whitehead, then Fell and Whitehead may very well know each other, completing a **triangle** in the visualization of three edges connecting Fox, Fell, and Whitehead. The number of these enclosed triangles in the network can be used to find clusters and communities of individuals that all know each other fairly well.

One way of measuring triadic closure is called **clustering coefficient** because of this clustering tendency, but the structural network measure you will learn is known as **transitivity**.11 Transitivity is the ratio of all triangles over all possible triangles. A possible triangle exists when one person (Fox) knows two people (Fell and Whitehead). So transitivity, like density, expresses how interconnected a graph is in terms of a ratio of actual over possible connections. Remember, measurements like transitivity and density concern likelihoods rather than certainties. All the outputs of your Python script must be interpreted, like any other object of research. Transitivity allows you a way of thinking about all the relationships in your graph that may exist but currently do not.

In [ ]:
triadic_closure = nx.transitivity(g)
print(f"Triadic closure: {round(triadic_closure, 6)}")

Also like density, transitivity is scaled from 0 to 1, and you can see that the network’s transitivity is about 0.1694, somewhat higher than its 0.0248 density. Because the graph is not very dense, there are fewer possible triangles to begin with, which may result in slightly higher transitivity. That is, nodes that already have lots of connections are likely to be part of these enclosed triangles. To back this up, you’ll want to know more about nodes with many connections.

### Centrality


In network analysis, measures of the importance of nodes are referred to as **centrality** measures. Because there are many ways of approaching the question “Which nodes are the most important?” there are many different ways of calculating centrality. Here you’ll learn about three of the most common centrality measures: degree, betweenness centrality, and eigenvector centrality.

##### Degree

Degree is the simplest and the most common way of finding important nodes. A node’s degree is the sum of its edges. If a node has three lines extending from it to other nodes, its degree is three. 

The nodes with the highest degree in a social network are the people who know the most people. These nodes are often referred to as hubs, and calculating degree is the quickest way of identifying hubs.

All of the centrality commands you’ll learn in this section produce dictionaries in which the keys are nodes and the values are centrality measures. That means they’re ready-made to add back into your network as a node attribute, like you did in the last section. 

In [ ]:
degree_dict = dict(g.degree(g.nodes()))
nx.set_node_attributes(g, degree_dict, 'degree')

In [ ]:
print(g.nodes['William Penn'])

In [ ]:
from operator import itemgetter

In [ ]:
sorted_degree = sorted(degree_dict.items(), key=itemgetter(1), reverse=True)

In [ ]:
print("Top 20 nodes by degree:")
for d in sorted_degree[:20]:
    print(d)

In this case almost all of the hubs are founders of the religion or otherwise important political figures. Thankfully there are other centrality measures that can tell you about more than just hubs. 

##### Eigenvector Centrality

Eigenvector centrality is a kind of extension of degree—it looks at a combination of a node’s edges and the edges of that node’s neighbors. Eigenvector centrality cares if you are a hub, but it also cares how many hubs you are connected to. It’s calculated as a value from 0 to 1: the closer to one, the greater the centrality. Eigenvector centrality is useful for understanding which nodes can get information to many other nodes quickly.

##### Betweenness Centrality

Betweenness centrality is a bit different from the other two measures in that it doesn’t care about the number of edges any one node or set of nodes has. Betweenness centrality looks at all **the shortest paths that pass through a particular node** (see above). To do this, it must first calculate every possible shortest path in your network, so keep in mind that betweenness centrality will take longer to calculate than other centrality measures (but it won’t be an issue in a dataset of this size). Betweenness centrality, which is also expressed on a scale of 0 to 1, is fairly good at finding nodes that connect two otherwise disparate parts of a network. If you’re the only thing connecting two clusters, every communication between those clusters has to pass through you. In contrast to a hub, this sort of node is often referred to as a broker. Betweenness centrality is not the only way of finding brokerage (and other methods are more systematic), but it’s a quick way of giving you a sense of which nodes are important not because they have lots of connections themselves but because they stand between groups, giving the network connectivity and cohesion.

In [ ]:
betweenness_dict = nx.betweenness_centrality(g) # Run betweenness centrality
eigenvector_dict = nx.eigenvector_centrality(g) # Run eigenvector centrality

# Assign each to an attribute in your network
nx.set_node_attributes(g, betweenness_dict, 'betweenness')
nx.set_node_attributes(g, eigenvector_dict, 'eigenvector')

In [ ]:
sorted_betweenness = sorted(betweenness_dict.items(), key=itemgetter(1), reverse=True)

print("Top 20 nodes by betweenness centrality:")
for b in sorted_betweenness[:20]:
    print(b)

In [ ]:
sorted_betweenness = sorted(eigenvector_dict.items(), key=itemgetter(1), reverse=True)

print("Top 20 nodes by eigenvector centrality:")
for b in sorted_betweenness[:20]:
    print(b)

What if you want to know which of the high betweenness centrality nodes had low degree? That is to say: which high-betweenness nodes are unexpected?

In [ ]:
#First get the top 20 nodes by betweenness as a list
top_betweenness = sorted_betweenness[:20]

#Then find and print their degree
for tb in top_betweenness: # Loop through top_betweenness
    degree = degree_dict[tb[0]] # Use degree_dict to access a node's degree, see footnote 2
    print("Name:", tb[0], "| Betweenness Centrality:", tb[1], "| Degree:", degree)

You can confirm from these results that some people, like Leavens and Penington, have high betweenness centrality but low degree. This could mean that these women were important brokers, connecting otherwise disparate parts of the graph. You can also learn unexpected things about people you already know about—in this list you can see that Penn has lower degree than Quaker founder George Fox, but higher betweenness centrality. That is to say, simply knowing more people isn’t everything.

### Community Detection